### HELPFUL NOTES

##### SET UP
###### Lesson 1

In [ ]:
## python -m venv fastapi_venv # install virtual environment

## fastapi_venv\Scripts\activate.bat # activate virtual environment first step
## fastapi_venv\Scripts\activate # second step

## pip install fastapi # install api  library
## pip install uvicorn # install server library

## uvicorn main:app --reload # run in cmd to run the main in a browser...it sets up a server using uvicorn. 
# You must indicate the file path to what you want to run and the object you want to run.
# The --reload reruns the server every time we make a change to the main.

##### In FASTAPI the documentation is automatically generated for the endepoints that we define in our code.

In [ ]:
# just add /docs in the link of the browser to check it out (used for testing ussually)
# or /redoc  for a different kind of documentation (used for readable documentation)

In [ ]:
# The order af methods dictates the priority of execution if they have similar paths

# Routers

In [ ]:
# Big apps need to be organised using routers

In [ ]:
# pip install -r requirements.txt

## Boilerplate code

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL="sqlite:///./fastapi-practice.db"

engine= create_engine(SQLALCHEMY_DATABASE_URL,connect_args={"check_same_thread": False})

SessionLocal = sessionmaker(autocommit=False,autoflush=False,bind=engine)

Base= declarative_base()

# GENERATE SECRETE UNIQUE KEY

In [ ]:
# Run in powershell
## [guid]::NewGuid().ToString("N")

# Authentication

In [ ]:
# To authenticate you need a function to create the token like:

from fastapi.security import OAuth2PasswordBearer
from typing import Optional
from datetime import datetime, timedelta
from jose import jwt

oauth2_schema = OAuth2PasswordBearer(tokenUrl='token')

SECRET_KEY='0146fea9e7e7454dbf97d7cba24d8b0a'

ALGORITHM='HS256'

ACCESS_TOKEN_EXPIRE_MINUTES=30

def create_access_token(data: dict, expires_delta: Optional[timedelta]=None):
    to_encode=data.copy()
    if expires_delta:
        expire=datetime.utcnow()+expires_delta
    else:
        expire=datetime.utcnow()+timedelta(minutes=15)
    to_encode.update({"exp":expire})
    encoded_jwt=jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

# And a function to authenticate and return the token using it, like:

from  auth import oauth2
from fastapi import APIRouter,HTTPException,status
from fastapi.param_functions import Depends
from sqlalchemy.orm.session import Session
from db.database import get_db
from fastapi.security.oauth2 import OAuth2PasswordRequestForm
from db import models
from db.hash import Hash


router= APIRouter(
    tags=["authentication"]
)

@router.post('/token')
def get_token(request:OAuth2PasswordRequestForm = Depends(),db: Session =Depends(get_db)):
    user=db.query(models.DbUser).filter(models.DbUser.username==request.username).first()
    if not user:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Invalid credentials")
    if not Hash.verify(user.password,request.password):
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND,detail="Incorrect password")
    
    access_token=oauth2.create_access_token(data={'sub':user.username})

    return {
        'access_token':access_token,
        'token_type':'bearer',
        'user_id': user.id,
        'username':user.username
    }

## Install Deta